Demonstrate a frequency scan of a simulated resonance.

This follows the steps in ../zcu216/mkids_2x2_kidsim_v2.ipynb, but should work in general on all platforms.
The parameter "bitfile" is set to select the firmware used, and it should work with all available firmwares.
Note that the previous demo_00_init.ipynb show how to find available firmwares on the platform.


For this notebook, we use the "Dual 0" chain for mkids software.  I

On the ZCu216 it uses these connections:
* DAC output on DAC Tile = 2, DAC Ch = 0, which is the connector 0_230, on JHC3
* ADC input on ADC Tile = 2, ADC Ch = 0, which is the connector 0_226, on JHC7

We use the "Sim 0" chain to simulate the resonance.  It uses these connections:
* DAC output on DAC Tile = 3, DAC Ch = 0, which is the connector 0_231, on JHC3
* ADC input on ADC Tile = 3, ADC Ch = 0, which is the connector 0_227, on JHC7

These are connected through the low-frequency Baluns.

On the ZCU11 it uses these connections:

Well you have a few options because there are 2 kid chains and 2 emulation chains. I was using this:

 

DAC 229_CH0 -> ADC 224_CH1

DAC 229_CH1 -> ADC 224_CH0

 

In [ ]:
import sys
sys.path.append('../../../qick/qick_lib/')
sys.path.append('../soft/')
import Scan
from mkids import *
import matplotlib.pyplot as plt


In [ ]:
board = Scan.getBoard()
if board == 'zcu111':
    bitfile = 'mkids_2x2_kidsim_v1'
    fMixerSimu = 1024
    fResonance = 805
    fMixerKids = 1024
    fc = 800
    df = 20

else:
    raise Exception("Do not know what to do with %s"%board)
    

In [ ]:
scan = Scan.Scan(bitfile)

In [ ]:
# Confirm what we have available
print("bitfile =",bitfile)
print(scan.soc)


In [ ]:
# Get the simulation chain.
#simu = SimuChain(scan.soc, simu=scan.soc['simu'][0])
simu = scan.simuChain

# Set quantization.
simu.analysis.qout(3)
simu.synthesis.qout(3)

# Set mixer.

simu.set_mixer_frequency(fMixerSimu)

# Disable all resonators.
simu.alloff()


# Enable one resonator 
print("fResonance = {} MHz".format(fResonance))
simu.enable(fResonance)


In [ ]:
# Get the KIDS chain
kids = scan.kidsChain
kids.set_mixer_frequency(fMixerKids)


In [ ]:
#######################
### Frequency Sweep ###
#######################
kids.analysis.qout(2)
kids.synthesis.qout(2)
f,a,phi=kids.sweep(fc-df/2,fc+df/2,N=500,g=0.1)

In [ ]:
plt.figure(dpi=150)
plt.plot(f,20*np.log10(a/max(a)))
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");

plt.figure(dpi=150)
plt.plot(f,phi)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");